In [76]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png','retina'}

In [ ]:
#다중출력여부
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 텐서플로우 튜토리얼 - 기본기 연습 및 간단한 모델
- 튜토리얼을 제공하는 깃허브 https://github.com/golbin/TensorFlow-Tutorials

# ◆ NeuralNetwork Basic

### Word2Vec
- 자연어 분석에 매우 중요하개 사용되는 Word2Vec모델 구현

In [ ]:
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

#한글폰트 적용
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)
#음수처리
mpl.rcParams['axes.unicode_minus'] = False

# 단어 벡터를 분석해볼 임의의 문장
sentences = ["나 고양이 좋다",
             "나 강아지 좋다",
             "나 동물 좋다",
             "강아지 고양이 동물",
             "여자친구 고양이 강아지 좋다",
             "고양이 생선 우유 좋다",
             "강아지 생선 싫다 우유 좋다",
             "강아지 고양이 눈 좋다",
             "나 여자친구 좋다",
             "여자친구 나 싫다",
             "여자친구 나 영화 책 음악 좋다",
             "나 게임 만화 애니 좋다",
             "고양이 강아지 싫다",
             "강아지 고양이 좋다"]

# 문장을 전부 합친 후 공백으로 단어들을 나누고 고유한 단어들로 리스트를 만든다.
word_sequence = " ".join(sentences).split()
word_list = " ".join(sentences).split()
word_list = list(set(word_list))
# 문자열로 분석하는 것 보다, 숫자로 분석하는 것이 훨씬 용이함
# 리스트에서 문자들의 인덱스를 뽑아 사용하기 위해서
# 이를 표현하기 위한 연관 배열과, 단어 리스트에서 단어를 참조할 수 있는 인덱스 배열을 만든다
word_dict = {w: i for i, w in enumerate(word_list)}

# 윈도우 사이즈를 1로하는 skip-gram모델을 만든다
# 예) 나 게임 만화 애니 좋다
#   -> ([나, 만화], 게임), ([게임, 애니], 만화), ([만화, 좋다], 애니)
#   -> (게임, 나), (게임, 만화), (만화, 게임), (만화, 애니), (애니, 만화), (애니, 좋다)
skip_grams=[]

for i in range(1, len(word_sequence)-1):
    # (context, target) : ([target index -1, target_index +1], target)
    # 스킵그램을 만든 후, 저장은 단어의 고류 번호(index)로 저장
    target = word_dict[word_sequence[i]]
    context = [word_dict[word_sequence[i-1]],word_dict[word_sequence[i+1]]]
    
    # (target, context[0]),(target, context[1])...
    for w in context:
        skip_grams.append([target, w])

# skip-gram 데이터에서 무작위로 데이터를 뽑아 입력값과 출력값의 배치 데이터를 생성하는 함수
def random_batch(data,size):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(data)),size, replace=False)
    
    for i in random_index:
        random_inputs.append(data[i][0]) # target
        random_labels.append([data[i][1]]) #context word
        
    return random_inputs, random_labels

#########=======================================
# 옵션 설정
######
# 학습을 반복할 횟수
training_epoch = 1000
# 학습률
learning_rate = 0.1
# 한번에 학습할 데이터의 크기
batch_size = 20
# 단어 벡터를 구성할 임베딩 차원의 크기
# 이 예제는 x,y그래프로 표현하기 쉽도록 2개값만 출력
embedding_size = 2
# word2ved 모델 학습을 위한 nce_loss 함수에서 사용하기 위한 샘플링 크기
# batch_size 보다 작아야 한다.
num_sampled = 15
# 총 단어갯수
voc_size = len(word_list)

#########=======================================
# 신경망 모델 구성
######
inputs = tf.placeholder(tf.int32, shape=[batch_size])
# tf.nn.nce_loss를 사용하려면 출력값을 이렇게 [batch_size, 1] 구성해야한다.
labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

# word2vec 모델의 결과값인 임베딩 벡터를 저장할 변수다.
# 총 단어 갯수와 임베딩 갯수를 크기로 하는 두개의 차원을 갖는다.
embeddings = tf.Variable(tf.random_uniform([voc_size, embedding_size],-1.0, 1.0))
# 임베팅 벡터의 차원에서 학습할 입력값에 대한 행들을 뽑아온다
# 예) embeddings     inputs    selected
#    [[1, 2, 3]  -> [2, 3] -> [[2, 3, 4]
#     [2, 3, 4]                [3, 4, 5]]
#     [3, 4, 5]
#     [4, 5, 6]]
selected_embed = tf.nn.embedding_lookup(embeddings, inputs)

# nce_loss 함수에서 사용할 변수들을 정의한다.
nce_weights = tf.Variable(tf.random_uniform([voc_size, embedding_size], -1.0, 1.0))
nce_biases = tf.Variable(tf.zeros([voc_size]))

# nce_loss 함수를 직접 구현하려면 매우 복잡하지만,
# 텐서플로우에서 제공하는 함수가 있으니 tf.nn.nce_loss함수를 사용하면됨
loss = tf.reduce_mean(
    tf.nn.nce_loss(nce_weights, nce_biases, labels, selected_embed, num_sampled, voc_size))
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

#########==========================================
# 신경망 모델 학습
######

with tf.Session() as sess:
    init= tf.global_variables_initializer()
    sess.run(init)
    
    for step in range(1, training_epoch + 1):
        batch_inputs, batch_labels = random_batch(skip_grams, batch_size)
        
        _, loss_val = sess.run([train_op, loss],
                              feed_dict={inputs:batch_inputs,
                                         labels:batch_labels})
        if step % 10 == 0:
            print("loss at step ",step, ": ",loss_val)
    
    # matplot 으로 출력하여 시각적으로 확인해보기 위해
    # 임베딩 벡터의 결과 값을 계한하여 저장한다.
    # with 구문 안에서는 sess.run 대신 간단히 eval() 함수를 사용할 수 있다.
    trained_embeddings = embeddings.eval()
    
#########=========================================
# 임베딩된 Word2Vec 결과 확인
# 결과는 해당 단어들이 얼마나 다른 단어와 인접해 있는지를 보여줍니다.
######    
for i, label in enumerate(word_list):
    x,y = trained_embeddings[i]
    plt.scatter(x,y)
    plt.annotate(label, xy=(x,y), xytext=(5,2),
                textcoords='offset points', ha='right', va='bottom')
    
plt.show()

### Deep NN
- 여러개의 신경망을 구성하는 방법 익혀보자

In [62]:
# 털과 날개가 있는지 없는지에 따라, 포유류인지 조류인지 분류하는 신경망 모델을 만들어봅니다.
# 신경망의 레이어를 여러개로 구성하여 말로만 듣던 딥러닝을 구성해 봅시다!
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array(
    [[0, 0], [1, 0], [1, 1], [0, 0], [0, 0], [0, 1]])

# [기타, 포유류, 조류]
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])

#########================================
# 신경망 모델 구성
######
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 첫번째 가중치의 차원은 [특성, 히든 레이어의 뉴런갯수] -> [2,10]으로 정한다.
W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
# 두번째 가중치의 차원을 [첫번째 히든 레이어의 뉴런 갯수, 분류갯수] -> [10,3] 으로 정한다.
W2 = tf.Variable(tf.random_uniform([10,3], -1., 1.))

# 편향을 각각 각 레이어의 아웃풋 갯수로 설정한다.
# b1은 히든 레이어의 뉴런 갯수로, b2는 최종 결과값 즉, 분류갯수인 3으로 설정
b1 = tf.Variable(tf.zeros([10]))
b2 = tf.Variable(tf.zeros([3]))

# 신경망의 히든 레이어에 가중치 W1과 편향 b1을 적용
L1 = tf.add(tf.matmul(X, W1),b1)
L1 = tf.nn.relu(L1)

# 최종적인 아웃풋을 계산한다
# 히든레이어에 두번째 가중치 W2와 편향 b2를 적용하여 3개의 출력값을 만들어낸다
model = tf.add(tf.matmul(L1, W2),b2)

# 덴서플로우에서 기본적으로 제공되는 크로스 엔트로피 함수를 이용해
# 복잡한 수식을 사용하지 않고도 최적화를 위한 비용함수를 다음처럼 간단하게 적용할 수 있음
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)


#########================================
# 신경망 모델 학습
######
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(100):
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    
    if (step + 1)% 10 == 0:
        print(step+1, sess.run(cost, feed_dict={X:x_data, Y:y_data}))
        
        

#########================================
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model,1)
target = tf.arg_max(Y,1)
prediction = tf.argmax(model,1)
target = tf.argmax(Y,1)
print('예측값:', sess.run(prediction, feed_dict={X:x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy*100, feed_dict={X:x_data, Y:y_data}))

10 1.10388
20 0.866431
30 0.674503
40 0.533089
50 0.42709
60 0.348081
70 0.287232
80 0.238058
90 0.197017
100 0.163432
예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


### Classification
- 신경망을 구성하여 간단한 분류 모델을 만들어보자

In [61]:
# 털과 날개가 있는지 없는지에 따라, 포유류인지 조류인지 분류하는 신경망 모델을 만들어본다.
import tensorflow as tf
import numpy as np

# [털, 날개]
x_data = np.array([[0,0],[1,0],[1,1],[0,0],[0,0],[0,1]])

# [기타, 포유류, 조류]
# 다음과 같은 형식은 Oh형식의 데이터라고 함 one-hot 데이터
y_data = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1],
    [1,0,0],
    [1,0,0],
    [0,0,1]
])

#########================================
# 신경망 모델 구성
#########
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

# 신경망은 2차원으로 [입력층(특성), 출력층(레이블)] -> [2,3]으로 정한다.
W = tf.Variable(tf.random_uniform([2,3],-1., 1.))

# 편향을 각각 레이어의 아웃풋 갯수로 설정
# 편향은 아웃풋의 갯수, 즉 최종 결과값의 분류 갯수인 3으로 설정
b = tf.Variable(tf.zeros([3]))

# 신경망에 가중치 W와 편향 b를 적용
L = tf.add(tf.matmul(X,W),b)
# 가중치와 편향을 이용해 계산한 결과값에
# 텐서플로우에서 기본적으로 제공하는 활성화 함수인 ReLU함수 적용
L = tf.nn.relu(L)

# 마지막으로 softmax함수를 이용하여 출력값을 사용하기 쉽게 만든다.
# softmax함수는 다음처럼 결과값을 전체합이 1인 확률로 만들어주는 함수
# 예) [8.04, 2.76, -6.52] -> [0.53 0.24 0.23]
model = tf.nn.softmax(L)

# 신경망을 최적화하기 위한 비용 함수를 작성
# 각 개별 결과에 대한 합을 구한뒤 평규을 내는 방식을 사용!
# 전체 합이 아닌, 개별 결과를 구한 뒤 평균을 내는 방식을 사용하기 위해 axis 옵션 사용
# (axis 옵션이 없으면 -1.09처럼 총합인 스칼라값으로 출력됨)
#        Y         model         Y * tf.log(model)   reduce_sum(axis=1)
# 예) [[1 0 0]  [[0.1 0.7 0.2]  -> [[-1.0  0    0]  -> [-1.0, -0.09]
#     [0 1 0]]  [0.2 0.8 0.0]]     [ 0   -0.09 0]]
# 즉, 이것은 예측값과 실제값 사이의 확률 분포의 차이를 비용으로 계산한 것이며,
# 이것을 Cross-Entropy라고 한다.
cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(model),axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost)


#########===============================
# 신경망 모델 학습
#########
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for step in range(3000): ## 이게 epoch와 같은건가? 100보다 30이 더 설득력있는데.. 그냥 트레이닝 스텝인듯.
    sess.run(train_op, feed_dict={X:x_data, Y:y_data})
    
    if (step+1)%10==0: # 학습상황을 프린트로 볼 수 있다.
        print(step+1, sess.run(cost, feed_dict={X:x_data, Y:y_data}))
        
#########===============================
# 결과확인
# 0:기타 1:포유류 2:조류
#########
# tf.argmax: 예측값과 실제값의 행렬에서 tf.argmax를 이용해 가장 큰 값을 가져온다.
# 예) [[0 1 0] [1 0 0]] -> [1 0]
#    [[0.2 0.7 0.1] [0.9 0.1 0.]] -> [1 0]

prediction = tf.argmax(model,1)
target = tf.argmax(Y,1)
print('예측값:', sess.run(prediction, feed_dict={X:x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy*100, feed_dict={X:x_data, Y:y_data}))


10 1.04917
20 1.04776
30 1.0465
40 1.04516
50 1.04391
60 1.04267
70 1.04139
80 1.0402
90 1.03936
100 1.03854
110 1.03775
120 1.03698
130 1.03619
140 1.03545
150 1.03467
160 1.034
170 1.03321
180 1.0325
190 1.0318
200 1.03108
210 1.0304
220 1.02977
230 1.02942
240 1.02909
250 1.02873
260 1.02849
270 1.02806
280 1.02775
290 1.0274
300 1.02708
310 1.02674
320 1.02645
330 1.02609
340 1.02575
350 1.02546
360 1.02511
370 1.02477
380 1.02448
390 1.02414
400 1.02391
410 1.02351
420 1.02318
430 1.0229
440 1.02257
450 1.02233
460 1.02196
470 1.02163
480 1.02136
490 1.02103
500 1.02072
510 1.02045
520 1.02012
530 1.01981
540 1.01955
550 1.01922
560 1.01891
570 1.01868
580 1.01834
590 1.01803
600 1.01772
610 1.01752
620 1.01716
630 1.01687
640 1.01661
650 1.01631
660 1.01601
670 1.01577
680 1.01552
690 1.01518
700 1.01489
710 1.01464
720 1.01436
730 1.01408
740 1.01377
750 1.01349
760 1.01329
770 1.01296
780 1.01268
790 1.01241
800 1.01219
810 1.01193
820 1.01162
830 1.01134
840 1.01107
850 1.0108

---
# ◆ Tensorflow basic

### Linear Regression
- 단순한 선형회귀모형을 만들어보자

In [13]:
# X와 Y의 상관관계를 분석하는 기초적인 선형회귀모델 만들기

import tensorflow as tf

x_data = [1,2,3]
y_data = [1,2,3]

W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

# name : 나중에 텐서보드등으로 값의 변화를 추적하거나 살펴보기 쉽게 하기 위해 이름을 붙임
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")
print(X)
print(Y)

# 상관관계 분석을 위산 가설수실을 작성합니다.
# y = W*x+b
# W와 X가 행렬이 아님 ==> tf.matmul이 아닌 기본곱셈 사용
hypothesis = W * X + b

# 손실함수를 작성
# mean(h-y)^2 : 예측값과 실제값의 거리를 비용함수로 정한다
cost = tf.reduce_mean(tf.square(hypothesis-Y))

# 텐서 플로우에 기본적으로 포함되어있는 함수를 이용, 경사하강법 최적화를 수행한다.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

# 비용을 최소화 하는것이 목표
train_op = optimizer.minimize(cost)


# 세션을 생성하고 초기화
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 최적화를 100번 수행한다.
    for step in range(100):
        # sess.run을 통해 train_op와 cost그래프를 계산한다.
        # 이 때, 가설 수식에 넣어야 할 실제값을 feed_dict을 통해 전달한다.
        _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y:y_data})
        
        print(step, cost_val, sess.run(W), sess.run(b))
        
    # 최적화가 완료된 모델에 테스트 값을 넣고 결과가 잘 나오는지 확인
    print("\n===Test===")
    print("X: 5, Y:", sess.run(hypothesis, feed_dict={X:5}))
    print("X: 2.5, Y:", sess.run(hypothesis, feed_dict={X: 2.5}))


Tensor("X_4:0", dtype=float32)
Tensor("Y_4:0", dtype=float32)
0 0.791301 [ 1.02837539] [ 0.03801347]
1 0.00951704 [ 0.98668629] [ 0.01906063]
2 0.000175426 [ 0.99148816] [ 0.02057398]
3 6.09058e-05 [ 0.99120295] [ 0.01986391]
4 5.67442e-05 [ 0.99146795] [ 0.01940995]
5 5.40333e-05 [ 0.99166721] [ 0.01894079]
6 5.14672e-05 [ 0.9918682] [ 0.01848576]
7 4.90219e-05 [ 0.99206358] [ 0.01804134]
8 4.66929e-05 [ 0.99225438] [ 0.01760764]
9 4.44753e-05 [ 0.99244052] [ 0.01718434]
10 4.23629e-05 [ 0.99262232] [ 0.01677127]
11 4.03505e-05 [ 0.99279964] [ 0.01636809]
12 3.84347e-05 [ 0.99297273] [ 0.01597461]
13 3.66091e-05 [ 0.99314171] [ 0.01559062]
14 3.48689e-05 [ 0.99330652] [ 0.01521581]
15 3.32131e-05 [ 0.99346745] [ 0.01485004]
16 3.16348e-05 [ 0.99362445] [ 0.01449303]
17 3.01326e-05 [ 0.99377775] [ 0.01414466]
18 2.87015e-05 [ 0.9939273] [ 0.01380462]
19 2.73377e-05 [ 0.99407327] [ 0.01347278]
20 2.604e-05 [ 0.99421579] [ 0.01314892]
21 2.48024e-05 [ 0.99435478] [ 0.01283281]
22 2.36242

### Variable
- 텐서플로우의 플레이스 홀더와 변수의 개념을 익히자

In [5]:
import tensorflow as tf

#placeholder : 계산을 실행할 때 입력값을 받는 변수로 사용한다.
#None은 크기가 정해지지 않았음을 의미함
X = tf.placeholder(tf.float32, [None, 3])
print(X)

# X플레이스홀더에 넣을값
# 플레이스홀더에서 설정한 것처럼, 두번째 차원의 요소갯수는 3개
x_data = [[1,2,3],[4,5,6]]

#Variable : 그래프를 계산하면서 최적화 할 변수들. 이 값이 신경망을 좌우하는 값
# random_normal : 각 변수들의 초기값을 정규분포랜덤값으로 초기화
W = tf.Variable(tf.random_normal([3,2]))
b = tf.Variable(tf.random_normal([2,1]))

# 입력값과 변수들을 계산할 수식을 작성
# tf.matmul 처럼 mat*로 되어있는 함수로 행렬계산을 수행
expr = tf.matmul(X,W) + b

sess = tf.Session()
# 위에서 설정한 Variable들의 값을 초기화 해야됨
# 처음에 tf.global_variables_initializer 를 한 번 실행해야함
sess.run(tf.global_variables_initializer())

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("===expr====")
print(sess.run(expr, feed_dict={X:x_data}))
# expr 수식에는 X라는 입력값이 필요
# 따라서 expr 실행시 이 변수에 대한 실제 입력값을 넣어줘야함

sess.close()

Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)
=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[-0.291262   -0.11953487]
 [-0.17635864  0.3196032 ]
 [ 0.96932673 -1.17966092]]
=== b ===
[[-0.78580576]
 [ 1.48390293]]
===expr====
[[ 1.47819519 -3.80511713]
 [ 5.25302219 -4.47418642]]


### basic
- 텐서플로우의 연산의 개념과 그래프를 실행하는 방법을 익힙니다.

In [3]:
import tensorflow as tf

# constant : 상수임
hello = tf.constant('Hello,TensorFlow!')
print(hello)

a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a,b) # a+b로도 쓸 수 있다.
print(c)

# 위에서 변수와 수식들을 정의했지만, 실행이 정의한 시점에서 실행되는 것이 아님. 뭔가 스파크처럼 게으른 느낌
# Session 객제와 run 메소드를 사용할 때 비로소 계산실행.
# 따라서 모델을 구성하는 것과, 실행하는 것을 분리하여 프로그램을 깔끔하게 작성할 수 있음!!!
# 그래프를 실행할 세션을 구성
sess = tf.Session()

print(sess.run(hello))
print(sess.run([a,b,c]))

#세션닫기
sess.close()




Tensor("Const_6:0", shape=(), dtype=string)
Tensor("Add_2:0", shape=(), dtype=int32)
b'Hello,TensorFlow!'
[10, 32, 42]
